![Logo 1](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech1.jpg)
<div class="alert alert-block alert-info">
<h1> Praktyczne zastosowania chmury obliczeniowej </h1>
<h2> 2.1. <i>Platform as a Service</i>  [laboratoria]</h2> 
<h3> Patryk Żywica (2021)</h3>
</div>

![Logo 2](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech2.jpg)

# Wstęp do zajęć

Niniejsze zajęcia poświęcone są chmurze obliczeniowej w modelu PaaS (Platform as a Service). Podstawowe informacje na temat tego modelu zostały przedstawione na wcześniejszych zajęciach. Do kluczowych cech usług w tym modelu należą:
* brak konieczności zarządzania systemem operacyjnym i zainstalowanym środowiskiem uruchomieniowym
* wysoki stopień automatyzacji, znaczna część konfiguracji odbywa się poprzez ustawienia domyślne i konwencję
* konieczność dopasowania aplikacji do wymagań platformy uruchomieniowej
* ograniczony dostęp do pewnych zasobów np. brak możliwości zapisu plików, ograniczenie komunikacji do protokołu HTTP

Dostępnych jest wiele usług działających w tym modelu PaaS. Do najpopularniejszych należą:

* Google App Engine - historycznie pierwsza taka usługa, która przetrwała do dziś
* Amazon AWS Elastic Beanstalk - w pewnym sensie jest to nakładka na usługę AWS EC2
* Microsoft Azure App Service
* Salesforce Heroku - jeden z pierwszych i wciąż najpopularniejszy dostawca usług tego typu
* DigitalOcean App Platform
* Red Hat OpenShift PaaS
* The Mendix Low-Code Platform
* Dev Graph Engine Yard

W czasie zajęć poznamy platformę **Heroku**, która bardzo dobrze obrazuje możliwości oferowane przez model PaaS. Co ważne, w odróżnieniu od wielu dostawców, Heroku oferuje tylko usługę w modelu PaaS.

## Na czym polega PaaS

Model chmury obliczeniowej PaaS zakłada przeniesienie na dostawcę usługi odpowiedzialności za praktycznie wszystkie elementy środowiska, w którym działa aplikacja. Dostawca oprócz sprzętu i infrastruktury, dostarcza system operacyjny, bazy danych oraz środowisko uruchomieniowe dla naszej aplikacji. 

W modelu tym tworzona aplikacja najczęściej musi być odpowiednio przygotowana, aby mogła zostać uruchomiona w ściśle kontrolowanym środowisku. W początkowych etapach rozwoju był to chyba najczęściej podnoszony argument przeciw takiemu rozwiązaniu. Ścisłe związanie tworzonego oprogramowania z dostawcą usługi było postrzegane jako istotne ograniczenie i zagrożenie dla projektów informatycznych. Wraz z globalnym wzrostem wykorzystania usług chmury obliczeniowej wzrosła również akceptacja dla zwiększenia stopnia powiązania oprogramowania z dostawcą usług (ang. vendor lockin).

Z drugiej strony sam stopień powiązania aplikacji z platformą uległ znacznemu zmniejszeniu. Obecnie większość usług działających w modelu PaaS (np. Heroku), pozwala uruchomić niemal dowolną aplikację, napisaną w dowolnej technologii, nawet jeśli nie była ona projektowana specjalnie z myślą o wdrożeniu w chmurze.

## Jak działa Platform as a Service

W następującej części omówiona zostanie zasada działania usługi typu PaaS na przykładzie Heroku. Heroku jest bardzo reprezentatywne dla tego typu usług, dzięki czemu wiedzę tę będzie można zastosować do większości dostępnych usług tego typu.

Zasadę działania Heroku można przedstawić z kilku perspektyw:
1. Perspektywa działania aplikacji
1. Perspektywa budowania i wdrażania aplikacji
1. Perspektywa wirtualizacji i infrastruktury

### Perspektywa działania aplikacji

Aplikacja zbudowana z wykorzystaniem rozwiązania typu PaaS może składać się z wielu modułów. Każdy moduł może korzystać z innych usług i działać w inny sposób. Jednak cechą wspólną wszystkich aplikacji wdrażanych w modelu PaaS jest udostępnianie usług w protokole HTTP i to moduł realizujący tę funkcjonalność jest najczęściej uważany za główny dla danej aplikacji (nie rzadko stanowi też jedyny moduł aplikacji). 

> **Heroku**: Przykładowo w Heroku taki moduł nazywany jest `web` i tylko on może przyjmować żądania HTTP pochodzące z zewnątrz.

Usługa PaaS udostępnia nam adres *URL* naszej aplikacji. Ten adres nie prowadzi jednak bezpośrednio do naszego modułu głównego. Zamiast tego każde wysłane żądanie HTTP trafia do tak zwanego *routera*. Jest to centralny punkt usługi, którego zadanie polega na odnalezieniu właściwej aplikacji, do której kierowane jest zapytanie oraz przekazanie danych do odpowiedniej instancji modułu. 

> **Heroku**: Heroku posiada jeden centralny *router* w każdym z regionów, a wszystkie one korzystają z jednego punktu startowego (*entry point*): `herokuapp.com`.

*Router* realizuje również proces tak zwanego równoważenia obciążenia (ang. *load-balancing*). Polega on na przekazywaniu żądań równolegle do różnych kopii (instancji) tego samego modułu. W ten sposób aplikacja może obsłużyć dowolnie wiele wiadomości jednocześnie bez obawy o znaczący wzrost czasu odpowiedzi. Aby to było możliwe konieczne jest przestrzeganie kilku zasad już na etapie projektowania aplikacji:

1. Moduł musi być bezstanowy - nie powinien przechowywać żadnych informacji w pamięci czy na dysku. Całość stanu powinna być przechowywana w zewnętrznej usłudze (np. bazie danych).
1. Komunikacja z modułem musi być bezstanowa - każde żądanie do modułu musi zawierać wszystkie informacje niezbędne do jego realizacji (patrz założenia stylu REST).
1. Moduł musi być lekki - aby możliwe było szybkie uruchamianie nowych instancji oraz wyłączania tych już niepotrzebnych.

Skalowanie takich bezstanowych modułów może odbywać się automatycznie lub manualnie, w zależności od potrzeb.

> **Uwaga**: Więcej na temat zasad jakie powinien spełniać projekt zoptymalizowany pod wdrożenie w modelu PaaS przedstawia koncepcja [The Twelve-Factor App](https://12factor.net/pl/).

Gdy moduł główny naszej aplikacji otrzyma już żądanie HTTP, może przystąpić do jego obsługi. Kod naszej aplikacji jest uruchamiany w zależności od środowiska, w którym aplikacja została napisana. Można wyróżnić trzy główne sposoby wykonania tego kodu:

1. Aplikacja samodzielnie uruchamia serwer HTTP nasłuchujący na odpowiednim porcie
1. Aplikacja jest przygotowana zgodnie z założeniami zewnętrznego serwera HTTP
1. Aplikacja dostarczana jest w postaci zbudowanego obrazu Docker

Sposób pierwszy jest najpopularniejszy w przypadku języków kompilowanych (np. Go, Rust, Java), podczas gdy drugi jest najpopularniejszy w językach skryptowych (Python - uWSGI, Ruby - Unicorn). Sposób trzeci, jest stosunkowo młody, choć większość dostawców oferuje taką usługę, nie zawsze jest ona tak samo wspierana jak pierwsze dwa podejścia. Wybór sposobu uruchamiania aplikacji ma spore znaczenie na wymaganych zasobów. Przykładowo w rozwiązaniu drugim nie ma potrzeby, aby proces odpowiedzialny za moduł aplikacji był stale uruchomiony, podczas gdy w pozostałych modelach jest to niezbędne.

> **Heroku**: Potrafi automatycznie wykryć w jaki sposób uruchomić daną aplikację korzystając z bardzo wielu predefiniowanych profili.

> **Heroku**: Obsługuje wszystkie trzy sposoby uruchamiania aplikacji, są one też w pełni wspierane.

> **Heroku**: W ramach darmowego dostępu, wszystkie aplikacje hostowane na Heroku, są automatycznie usypiane, jeśli nie są aktywne (nie otrzymują żądań HTTP) przez określony czas.

Gdy odpowiedni proces aplikacji otrzymuje żądanie HTTP, obsługuje je zgodnie ze swoją implementacją. Co ważne może on w tym celu wykorzystywać:
* inne procesy w ramach tej samej aplikacji - przykładowo procesy robocze do wykonywania zadań w tle,
* usługi dodatkowe - w tej kategorii najczęściej znajdują się zarządzane bazy danych oraz zasoby dyskowe od dostawcy usługi PaaS,
* usługi zewnętrzne i zasoby Internetu - każda aplikacja może niemal dowolnie korzystać z zasobów dostępnych przez sieć Internet.

Na koniec obsługi żądania, zwracana jest odpowiedź HTTP, która przekazywana jest do *routera* i następnie do przeglądarki użytkownika aplikacji.

![Schemat przedstawia architekturę usługi typu PaaS. Zaznaczono poszczególne warstwy infrastruktury: zasilanie i dostęp do sieci, sprzęt i dyski, system operacyjny, środowisko uruchomieniowe. Twórca aplikacji dostarcza tylko kod modułów (A, B, C, D) realizujących logikę biznesową systemu, które są odseparowane od pozostałych części systemu. Również serwer HTTP oraz skalowanie horyzontalne jest zapewniane przez dostawcę. Aplikacja może swobodnie łączyć się z innymi zasobami w Internecie oraz korzystać z usług dodatkowych takich jak: przechowywanie plików, autoryzacja czy baza danych.](obrazy/paas-runtime.png)

Rysunek 1. Przykładowa architektura usługi typu PaaS. Dostawca zapewnia wszystko z wyjątkiem kodu aplikacji (moduły A, B, C i D).

## Perspektywa budowania i wdrażania aplikacji

Proces budowania i wdrażania aplikacji w wielu popularnych usługach typu PaaS oparty jest na repozytorium kodu. Aplikacja budowana jest po każdej zmianie wybranej gałęzi repozytorium.

> **Heroku**: W przypadku Heroku, należy przesłać zmiany na specjalne, tworzone dla każdej aplikacji repozytorium hostowane na serwerach Heroku.

Takie podejście do budowania aplikacji, wspomaga (a tak właściwie wymaga) wdrożenie modelu CI (ang. Continuous Integration, ciągła integracja). Budowanie aplikacji musi być w pełni zautomatyzowane i wykonywane w ściśle kontrolowanym środowisku. Platforma PaaS udostępnia schematy, według których budowane są aplikacje. Często też możliwe jest zdefiniowanie własnych schematów oraz budowanie aplikacji w dowolnym środowisku z wykorzystaniem kontenerów Docker.

> **Heroku**: Heroku obsługuje większość popularnych języków programowania i frameworków: Node.js, Java, Python, PHP, Go i [wiele innych](https://devcenter.heroku.com/articles/buildpacks). Aplikację można też zbudować korzystając z kontenerów.

Przykładowo w środowisku Python, zwyczajowo wszystkie zewnętrzne zależności powinny być opisane w pliku `requirements.txt`. Podczas zautomatyzowanego budowania aplikacji, odpowiedni schemat budowania dla języka Python, zainstaluje wszystkie wymagane zależności i dopiero wtedy przystąpi do budowania właściwej aplikacji.

Zbudowana aplikacja jest łączona z konfiguracją a następnie pakowana i nazywana (np. nazwą, wersją, numerem comita w repozytorium, czy bieżącą datą) tworząc *wydanie* (ang. release). Kolejne wydania aplikacji tworzą rejestr, do którego można tylko dodawać nowe elementy. W ten sposób zawsze istnieje możliwość powrotu do wersji wcześniejszej. Tak zdeponowane wydanie może następnie posłużyć do wykonania wdrożenia aplikacji. Jedna zbudowana aplikacja za sprawą różnych konfiguracji (np. testowa, produkcyjna) może prowadzić to wielu wydań. Każde wydanie może zostać uruchomione wielokrotnie czy to w ramach procesu skalowania, czy odtwarzania po awarii jednej z instancji.

![Schemat przedstawia proces budowania, wydawania i uruchamiania aplikacji w modelu PaaS. Proces rozpoczyna się od comita na repozytorium kodu. Kod jest następnie przesyłany do zautomatyzowanego systemu budowania, gdzie jest łączony z zależnościami oraz budowany. Tak zbudowanie oprogramowanie łączone jest następnie z konfiguracją tworząc wydanie. Wydanie aplikacji może zostać uruchomione w różnych środowiskach.](obrazy/paas-build.png)

Rysunek 2. Przykładowy schemat procesu budowania aplikacji na potrzeby usługi w modelu Paas.

## Perspektywa wirtualizacji i infrastruktury

Korzystając z usług w modelu PaaS, warto też być świadomym jak fizycznie zorganizowana jest infrastruktura, z której korzystamy. Przedstawione tu informacje są bardzo ogólne, gdyż przyjęte rozwiązania techniczne różnią się u poszczególnych dostawców. Można jednak przyjąć, że przedstawiona organizacja jest wspólna dla większości usług.

Przede wszystkim aplikacje uruchamiane w usłudze PaaS korzystają z w pełni zwirtualizowanego środowiska. Najczęściej jest to wirtualizacja kilku poziomowa. Na załączonym schemacie fizyczny serwer (o bardzo dużych zasobach obliczeniowych) jest hostem dla wielu maszyn wirtualnych. Pojedyncza maszyna wirtualna jest natomiast hostem dla bardzo wielu aplikacji wdrożonych w modelu PaaS. Każda z takich aplikacji może pracować we własnym zwirtualizowanym środowisku (np. kontenerze Docker). Całość połączona jest poprzez sieć wirtualną i zabezpieczona odpowiednio skonfigurowaną zaporą sieciową. *Router* jest odpowiedzialny za przesyłanie żądań HTTP do odpowiednich maszyn wirtualnych i dalej do kontenerów i aplikacji.

Przy przydzielaniu zasobów stosuje się bardzo często tak zwany over-provisioning. Polega to na udostępnianiu większej ilości wirtualnych zasobów niż jest fizycznie dostępne. Przyjmuje się założenie, że jest bardzo mało prawdopodobne, aby jednocześnie wszystkie aplikacje potrzebowały 100% zadeklarowanych zasobów. Oczywiście w skrajnym przypadku może to powodować przerwy w działaniu niektórych aplikacji oraz spadki wydajności. Stąd tak ważna jest konfiguracja skalowania i instancji zapasowych. Z drugiej strony takie podejście pozwala drastycznie zmniejszyć koszty usługi. W porównaniu z dedykowanym serwerem redukcja kosztów może sięgać nawet 100x, a w porównaniu z zasobami wirtualnymi, ale o dedykowanych zasobach 10x. Niektórzy dostawcy usług oferują wariant usługi w ramach, którego zasoby nie są współdzielone i/lub przydzielane wielokrotnie. Taka usługa jest znacznie droższa od standardowej, ale gwarantuje stabilną wydajność i dostępność nie zależnie od obciążenia zasobów usługodawcy.

> **Heroku**: Jak już zostało to powiedziane wcześniej, Heroku nie utrzymuje swojej własnej dedykowanej infrastruktury. Korzysta natomiast z zasobów AWS, w szczególności z maszyn wirtualnych EC2, które stanowią podstawę mocy obliczeniowej dostępnej dla aplikacji. Jedna maszyna wirtualna nawet najsłabsza, może bez problemu obsłużyć wiele aplikacji. W ten sposób wdrożenie aplikacji na Heroku może nadal być bardziej opłacalne niż bezpośrednio na AWS EC2. Ciekawe opracowanie na ten temat znajduje się [tu](https://christopher.xyz/2019/01/23/heroku-dyno-sizes.html).

![test](obrazy/paas-hardware.png)
Rysunek 3.

# Zadania do realizacji w czasie zajęć


## Konfiguracja środowiska

1. Zarejestruj się w usłudze Heroku korzystając z formularza pod adresem: <https://signup.heroku.com/>.

1. Zainstaluj narzędzie do zarządzania aplikacjami w chmurze Heroku:

    ```bash
    cd /dev/shm && wget https://cli-assets.heroku.com/heroku-linux-x64.tar.gz && tar xvf heroku-linux-x64.tar.gz
    ```
    > **Uwaga**: Oczywiście Heroku CLI możesz zainstalować w dowolnej lokalizacji w systemie, jeśli chcesz aby komenda `heroku` była dostępna globalnie w całym systemie zainstaluj ją w `/usr/local/bin`.

    > **Uwaga**: Katalog `/dev/shm` to dość specjalne miejsce w każdym współczesnym systemie linuxowym. Jest to dysk zlokalizowany w pamięci operacyjnej, w związku z czym jego zawartość jest czyszczona przy każdym wyłączeniu komputera. Ponadto pamiętaj, że umieszczając tam duże pliki ograniczasz ilość pamięci operacyjnej dostępnej dla systemu i innych procesów.

1. Zaloguj się w zainstalowanym narzędziu:

    ```bash
    /dev/shm/heroku/bin/heroku login
    ```

    Komanda ta może poprosić Cię o otwarcie w przeglądarce internetowej wyświetlonego linku w celu wykonania logowania do serwisu Heroku.

    > **Uwaga**: Dane logowania (login i hasło) nie zostaną zapisane na komputerze. Heroku (podobnie jak większość współczesnych serwisów dbających o bezpieczeństwo użytkowników) korzysta z tak zwanych *tokenów* dostępowych. Podając login i hasło, system weryfikuje Twoje uprawnienia, a następnie generuje specjalny token dostępowy, który następnie pozwala na dostęp do zasobów bez podawania/przechowywania hasła. Token może mieć ograniczone uprawnienia, a co najważniejsze można go unieważnić. Więcej informacji znajduje się w [dokumentacji Heroku](https://devcenter.heroku.com/articles/authentication).


## Przykładowa aplikacja

1. Sklonuj repozytorium Git zawierające przykładowy projekt startowy w Javie (używający frameworka Spring):


    ```bash
    cd ~
    git clone https://github.com/heroku/java-getting-started.git
    cd java-getting-started
    ```

1. Utwórz nową aplikację na Heroku i wyświetl informacje o niej:

    ```bash
    appName=$(/dev/shm/heroku/bin/heroku create --stack heroku-20 --region eu --json | jq -r '.name')
    /dev/shm/heroku/bin/heroku apps:info $appName
    ```
    
	Nowo utworzoną aplikację zobaczymy także w interfejsie webowym pod adresem: <https://dashboard.heroku.com/apps>.

    > **Uwaga**: Tak naprawdę wystarczyłaby komenda `heroku create` (ewentualnie z własną nazwą tworzonej aplikacji), reszta powyższego polecenia przechwytuje nazwę nowo utworzonego projektu i zapisuję ją w zmiennej powłoki do późniejszego użycia.

    > **Uwaga**: Tworząc aplikację nie podaliśmy jej nazwy, Heroku generuje je automatycznie jako zbitki dwóch losowych słów z języka angielskiego.

    > **Uwaga**: Parametr `--stack` określa bazowy obraz kontenera na podstawie, którego utworzona zostanie aplikacja. Więcej informacji w [dokumentacji](https://devcenter.heroku.com/articles/stack).

1. Polecenie `heroku create` nie tylko utworzyło aplikacje w infrastrukturze Heroku, ale jednocześnie zarejestrowało dodatkowe repozytorium docelowe (upstream) dla naszej aplikacji. Dzięki temu można bardzo łatwo wysłać kod aplikacji na serwery Heroku korzystając z narzędzia `git`:

    ```bash
    git push heroku main
    ```

	Aplikacja powinna być dostępna po podanym na końcu linkiem. Link można też uzyskać w interfejsie webowym, jest on stały dla danej aplikacji.

## Modyfikacja aplikacji

1. Dodamy teraz obsługę nowej ścieżki w naszej przykładowej aplikacji. Zmodyfikuj plik `src/main/java/com/example/Main.java` dodając:

    * w nagłówku pliku 
      ```java
      import org.springframework.web.bind.annotation.PathVariable;
      ```
    * wewnątrz klasy Main
      ```java
        @RequestMapping("/hello/{name}")
        String hello(@PathVariable(value="name") String name) {
          return "hello";
        }
      
        
      ```

1. Dodaj plik `src/main/resources/templates/hello.html` o następującej treści:

    ```html
    <!DOCTYPE html>
    <html xmlns:th="http://www.thymeleaf.org" th:replace="~{fragments/layout :: layout (~{::body},'db')}">
    
    <body>
      <div class="container">
        <h1>Hello <span th:text="${name}">name</span>!</h1>
      </div>
    </body>
    </html>
    ```

1. Utwórz nowy commit i wyślij zmiany do Heroku.

    ```bash
    git add src/main/resources/templates/hello.html
    git commit -a -m 'Hello'
    git push heroku main
    ```

    Po krótkiej chwili aplikacja powinna zostać podmieniona.

    > **Wskazówka**: Kompletne logi z procesu kompilacji i wdrażania aplikacji dostępne są w interfejsie webowym.

1. Aplikację można usunąć za pomocą interfejsu webowego lub z linii poleceń:

    ```bash
    /dev/shm/heroku/bin/heroku apps:delete $appName
    ```

## Aplikacja od zera w Pythonie

1. W tym zadaniu utworzymy aplikację od zera i uruchomimy ją w chmurze Heroku. Skorzystamy z gotowej przykładowej aplikacji wyświetlającej losowe strony.

1. Stwórz nowy katalog i wewnątrz niego utwórz puste repozytorium `git`:

    ```bash
    mkdir newApp
    cd newApp
    git init
    ```

1. Skopiuj do tego katalogu cztery pliki (`layout.jinja2`, `table.jinja2`, `lipsum.jinja2` i `main.py`) dołączone do materiałów do zajęć.

1. Tworzymy nową aplikację na Heroku i wysyłamy naszą aplikację:

    ```bash
    appName=$(/dev/shm/heroku/bin/heroku create --stack heroku-20 --region eu --json | jq -r '.name')
    git add .
    git commit -m 'Pierwsza wersja'
    git push heroku master
    ```

    Ostatnia linijka się nie powiedzie, bo Heroku nie wie co zrobić z wysyłanym kodem. Możesz też sprawdzić w interfejsie webowym co się stało.

1. Heroku stara się automatycznie wykryć język aplikacji i odpowiednio ją skompilować/zbudować. Pisząc własną aplikację warto poczytać dokładnie jak to robi. Większość języków/frameworków ma pewne konwencje nazywania plików czy wyszczególniania zależności i wystarczy się do nich stosować, by Heroku obsłużyło je poprawnie. 

    > **Uwaga**: Takie podejście znacząco ułatwia korzystanie z platformy, szczególnie początkującym użytkownikom. W dalszej części zajęć pokazane zostanie, jak przejąć pełnie kontroli nad Heroku i podjąć wszystkie decyzje samodzielnie.

    Obecność pliku `requirements.txt` jest sygnałem dla Heroku, że mamy do czynienia z aplikację napisaną w Pythonie, a w tym pliku są zależności do zainstalowania przez managera pakietów `pip`. Tworzymy więc w naszym projekcie plik `requirements.txt` o zawartości:

    ```python
    pyramid
    pyramid-jinja2
    pyramid-debugtoolbar
    faker
    uwsgi
    ```

	Następnie dodajemy go do repozytorium i wysyłamy do Heroku:

	```bash
	git add requirements.txt
    git commit -m 'requirements.txt'
    git push heroku master
    ```

    Teraz powinniśmy widzieć jak ściągane są zależności naszej aplikacji i na końcu zobaczymy link pod jakim nasza aplikacja 'działa'. Odwiedź stronę w przeglądarce, czy strona działa zgodnie z oczekiwaniami?

1. Logi naszej aplikacji można zawsze sprawdzić w interfejsie webowym. Można to też zrobić bezpośrednio z konsoli:

    ```bash
    /dev/shm/heroku/bin/heroku logs -a $appName
    ```

    Aplikacja co prawda się zbudowała, ale Heroku dalej nie wie jak ją uruchomić.

1. Aby poinstruować Heroku jak zbudowaną aplikację uruchomić, musimy utworzyć plik `Procfile` (uwaga na wielką literę) o zawartości:

	```
    web: uwsgi --http-socket=:$PORT --die-on-term --module=main:app
	```

    Mówimy w nim, że nasza aplikacja jest aplikacją webową i można ją uruchomić podanym poleceniem (to samo zadziałałoby lokalnie po utworzeniu odpowiedniego środowiska wirtualnego). Tutaj chcemy, by uWSGI nasłuchiwał na porcie o numerze pochodzącym ze zmiennej środowiskowej `PORT` (definiowanej przez Heroku).

	Po wysłaniu zmian do Heroku wszystko powinno już działać.

1. Aplikację możemy łatwo usunąć:

    ```bash
    /dev/shm/heroku/bin/heroku apps:delete $appName
    ```


## Większa kontrola nad Heroku

Heroku bardzo dobrze wykrywa domyślne ustawienia i konfiguracje. Czasem jednak to nie wystarcza. Struktura wdrażanej aplikacji może być zbyt złożona lub dostosowana pod inną infrastrukturę. W takiej sytuacji pomocą służą pliki konfiguracyjne Heroku:

* [Procfile](https://devcenter.heroku.com/articles/procfile) - samodzielnie sprawdza się dla najprostszych aplikacji, często działa w połączeniu z konfiguracją w `app.json`
* [app.json](https://devcenter.heroku.com/articles/app-json-schema) - pozwala definiować dodatkowe informacje na temat aplikacji oraz umożliwia zarządzanie dodatkami (np. bazą danych)
* [heroku.yml](https://devcenter.heroku.com/articles/build-docker-images-heroku-yml) - stworzony z myślą o wdrażaniu przy użyciu narzędzia `Docker`. Nie jest kompatybilny `Procfile`.


Przekształcimy teraz poprzednią aplikację tak aby uzyskać pełną kontrolę nad środowiskiem, w którym będzie działać. Wykorzystamy do tego narzędzie `Docker` i konfigurację z wykorzystaniem pliku `heroku.yml`.

1. Dodaj plik `heroku.yml` o następującej treści:

    ```yaml
    build:
        docker:
            web: Dockerfile
    run:
        web: uwsgi --http-socket=:$PORT --die-on-term --module=main:app
    ```
    Pierwsza część informuje Heroku w jaki sposób zbudować naszą aplikację, dla uzyskania pełnej elastyczności podajemy tylko nazwę pliku `Dockerfile` naszej aplikacji. W ten sposób możemy zrealizować dowolne operacje, skorzystać z dowolnych narzędzi.

    > **Wskazówka**: W sekcji `docker` można umieścić wiele pozycji, co umożliwi zbudowanie wielu obrazów docker.

    Druga część pliku odpowiada za określenie jak uruchomić naszą aplikację. Działa podobnie jak Procfile.

    > **Wskazówka**: W sekcji `run` można umieścić wiele pozycji, co umożliwi uruchomienie wielu procesów w ramach jednej aplikacji.

1. Utwórz plik `Dockerfile` o następującej treści

    ```dockerfile
    FROM python:3
    WORKDIR /usr/src/app

    COPY requirements.txt ./
    RUN pip install --no-cache-dir -r requirements.txt

    COPY . .
    ```

    Bazujemy tu na oficjalnym obrazie Python. Na początek kopiujemy plik `requirements.txt` i instalujemy wszystkie zależności, następnie kopiujemy resztę projektu. Nie podajemy dyrektyw `ENTRYPOINT` ani `CMD`, gdyż zostaną one zastąpione przez konfigurację z sekcji `run` pliku `heroku.yml`.



1. Usuwamy plik `Procfile`.

    ```bash
    rm Procfile
    ```

1. Zmieniamy *stack* naszej aplikacji w Heroku tak aby oparta była ona o kontener:

    ```bash
    /dev/shm/heroku/bin/heroku stack:set container
    ```

1. Dodajemy pliki do repozytorium i wysyłamy do Heroku. Nowa wersja strony powinna zostać zbudowana i uruchomiona. Zwróć uwagę na proces budowania, który obecnie pobiera i buduje odpowiednie obrazy dockerowe. Heroku nie korzysta już ze swoich domyślnych ustawień, zamiast tego buduje obraz docker zgodnie z dostarczaną specyfikacją.